<a href="https://colab.research.google.com/github/stephenbeckr/convex-optimization-class/blob/master/Demos/CVX_demo/tutorialSolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python solutions for `cvxpy` exercise

You can see what's in the output of `prob` by doing `vars(prob.solver_stats)`

The default solver depends on what kind of problem you're solving, but it's often `ECOS`.  See [advanced cvxpy settings](https://www.cvxpy.org/tutorial/advanced/index.html#setting-solver-options) for how to change accuracy options.  For `ECOS` the relevant options are `abstol` and `reltol` (defaults are 1e-8).

This notebook is for [https://github.com/stephenbeckr/convex-optimization-class/tree/master/Demos/CVX_demo](https://github.com/stephenbeckr/convex-optimization-class/tree/master/Demos/CVX_demo)

In [56]:
import numpy as np # np.array (and used internally in cvxpy)
import cvxpy as cvx
import sys

print("Using CVX version", cvx.__version__)
print(" and python version", sys.version)

A = np.array([[1, 6,11, 5,10, 4, 9, 3, 8, 2],
              [2, 7, 1, 6,11, 5,10, 4, 9, 3],
              [3, 8, 2, 7, 1, 6,11, 5,10, 4],
              [4, 9, 3, 8, 2, 7, 1, 6,11, 5],
              [5,10, 4, 9, 3, 8, 2, 7, 1, 6]])

y = np.array([1,2,3,4,5]).T

tol = 1e-12

def print_status(prob, x):
   print("Problem status: ", prob.status);
   print("Used the solver: ", 
         prob.solver_stats.solver_name, "with", 
         prob.solver_stats.num_iters, "iterations.")
   print("Optimal value:  ", prob.value)
   print("Optimal var:\n", x.value)

Using CVX version 1.0.31
 and python version 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


## Problem 1

In [31]:
x = cvx.Variable(10) # column vector with 10 elements

obj = cvx.Minimize(cvx.norm(x)) # cvx.norm defaults to the 2-norm
constraints = [cvx.norm(A*x-y) <= 0.1] # specify a list of constraints

prob = cvx.Problem(obj, constraints)
prob.solve(abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  ECOS with 12 iterations.
Optimal value:   0.2942164768158885
Optimal var:
 [ 0.08864833  0.13978232 -0.03833681  0.12955552 -0.04299758  0.11932872
 -0.04291765  0.10910192 -0.03608032  0.09887512]


## Problem 2

In [33]:
x = cvx.Variable(10)

obj = cvx.Minimize(cvx.norm(x)**2) # cvxpy objects implement the standard python ops
constraints = [cvx.norm(A*x-y) <= 0.1]

prob = cvx.Problem(obj, constraints)
prob.solve(abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  ECOS with 12 iterations.
Optimal value:   0.08656333522989969
Optimal var:
 [ 0.08864833  0.13978232 -0.03833684  0.12955552 -0.04299756  0.11932872
 -0.04291762  0.10910192 -0.03608035  0.09887512]


## Problem 3

In [35]:
x = cvx.Variable(10)

obj = cvx.Minimize(cvx.norm(x, p=1))
constraints = [cvx.norm(A*x-y) <= 0.1]

prob = cvx.Problem(obj, constraints)
prob.solve(abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  ECOS with 11 iterations.
Optimal value:   0.7876692193982971
Optimal var:
 [ 2.86980950e-14  5.51546207e-01 -5.62016863e-02  2.34152027e-13
 -6.22342549e-02  8.99424944e-14 -6.22342549e-02  5.32847508e-14
 -5.54528157e-02  3.74707520e-14]


## Problem 4

In [38]:
def get_problem1_dual_value():
  obj = cvx.Minimize(cvx.norm(x))
  constraints = [cvx.norm(A*x-y) <= 0.1]

  prob = cvx.Problem(obj, constraints)
  prob.solve(abstol=tol,reltol=tol)
  
  return constraints[0].dual_value

x = cvx.Variable(10)

# resolve problem 1 and return dual value for the constraint
l = get_problem1_dual_value()
print("dual variable: ", l)

obj = cvx.Minimize(cvx.norm(x) + l*cvx.norm(A*x-y))

prob = cvx.Problem(obj)
prob.solve(abstol=tol,reltol=tol)

# note that the solution is the same, but the optimal value is different,
# since for problem 1 we form the Lagrangian \|x\|_2 + \lambda(\|Ax-y\|_2-0.1)
print_status(prob, x)

# the optimal value for problem 1 should be
print("problem 1 optimal value: ", prob.value - 0.1*l)

dual variable:  0.07086588504529695
Problem status:  optimal
Used the solver:  ECOS with 10 iterations.
Optimal value:   0.3013030653202569
Optimal var:
 [ 0.08864831  0.1397823  -0.03833679  0.1295555  -0.04299758  0.11932871
 -0.04291765  0.10910191 -0.03608028  0.09887511]
problem 1 optimal value:  0.2942164768157272


## Problem 5

In [40]:
# x = cvx.Variable(5)  # we need to make this an explicit column vector
x = cvx.Variable((5,1))
ones = np.ones((10,1))

obj = cvx.Minimize(sum(cvx.norm(A-x@ones.T, axis=0))) # cvx.norm behaves like np.linalg.norm

prob = cvx.Problem(obj)
prob.solve(abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  ECOS with 15 iterations.
Optimal value:   63.95513865824523
Optimal var:
 [[4.53758566]
 [5.0590677 ]
 [5.64345808]
 [6.23527782]
 [6.78794077]]


## Problem 6

In [51]:
x = cvx.Variable((5,1)) # again, be careful here
ones = np.ones((10,1))

obj = cvx.Minimize(cvx.norm(A-x*ones.T))

prob = cvx.Problem(obj)
# ECOS solver won't solve this kind of thing. SCS is the new default
prob.solve(verbose=False,eps=tol) # ~1e-7 duality gap, but CVXOPT gets a singular KKT system
print_status(prob, x)
prob.solve(verbose=False, solver='CVXOPT', kktsolver='robust')
print_status(prob, x)

Problem status:  optimal
Used the solver:  SCS with 221 iterations.
Optimal value:   14.392200222548352
Optimal var:
 [[5.90534845]
 [5.79946083]
 [5.69654806]
 [5.59838801]
 [5.50500124]]
Problem status:  optimal
Used the solver:  CVXOPT with None iterations.
Optimal value:   14.392200216477523
Optimal var:
 [[5.9       ]
 [5.79999997]
 [5.69999998]
 [5.59999995]
 [5.49999996]]


## Problem 7

In [53]:
# X = cvx.Variable(5,10) # old syntax
X = cvx.Variable((5,10))

obj = cvx.Minimize(cvx.norm(X-A, 'fro'))
constraints = [ np.ones((5,)).T*X*np.ones((10,)) == 1. ]

prob = cvx.Problem(obj, constraints)
prob.solve()

print_status(prob, X)

Problem status:  optimal
Used the solver:  ECOS with 5 iterations.
Optimal value:   40.16366509797013
Optimal var:
 [[-4.67999426  0.31999963  5.31999352 -0.67999914  4.31999475 -1.67999792
   3.31999597 -2.6799967   2.31999719 -3.67999548]
 [-3.67999548  1.31999841 -4.67999426  0.31999963  5.31999352 -0.67999914
   4.31999475 -1.67999792  3.31999597 -2.6799967 ]
 [-2.6799967   2.31999719 -3.67999548  1.31999841 -4.67999426  0.31999963
   5.31999352 -0.67999914  4.31999475 -1.67999792]
 [-1.67999792  3.31999597 -2.6799967   2.31999719 -3.67999548  1.31999841
  -4.67999426  0.31999963  5.31999352 -0.67999914]
 [-0.67999914  4.31999475 -1.67999792  3.31999597 -2.6799967   2.31999719
  -3.67999548  1.31999841 -4.67999426  0.31999963]]


## Problem 8

In [ ]:
B = A[:,0:5]
X = cvx.Variable(5,5) # could use Semidef or Symmetric here instead

obj = cvx.Minimize(cvx.norm(X-B, 'fro'))
constraints = [ X == X.T, X >> 0 ] # X is PSD

prob = cvx.Problem(obj, constraints)
prob.solve()

print_status(prob, X)